In [2]:
from pathlib import Path
import sys
parent = Path.cwd().resolve().parents[0]
sys.path.append(str(parent))
from SMABacktestClass import SMAVectorBacktest as SMA
import pandas as pd
import requests
import re

In [ ]:
start_date = '2022-01-01'
end_date = '2025-01-01'

# We need to ensure that, for whatever backtest window we choose, price data is available for every stock so the performance comparison is fair and measured over the same exact period. 
# Therefore, the earliest valid start date should be the latest common date for which data for all 101 assets of the Nasday 100 are available.

# Getting the yahoo finance tickers for all stocks in the Nasdaq 100

In [4]:
url = "https://en.wikipedia.org/wiki/Nasdaq-100"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

html = requests.get(url, headers=headers, timeout=30).text
tables = pd.read_html(html)

components = next(df for df in tables if "Ticker" in df.columns)
tickers = components["Ticker"].tolist()

# Yahoo Finance uses '-' instead of '.' for tickers like BRK.B -> BRK-B
tickers = [t.replace(".", "-") for t in tickers]

print(len(tickers))
print(tickers[:15])


101
['ADBE', 'AMD', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'APP', 'ARM', 'ASML', 'AZN']


C:\Users\luciu\AppData\Local\Temp\ipykernel_19168\2793512619.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [5]:
summary_dict = {}
for ticker in tickers: 
    bt = SMA(ticker,SMA1=10,SMA2=50,start=start_date,end=end_date)
    summary_dict[ticker] = bt.run_strategy()

print(summary_dict)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ADBE': 'buy&hold: 5.46%, strategy: -43.05%, alpha: -48.51%', 'AMD': 'buy&hold: 10.48%, strategy: -27.53%, alpha: -38.01%', 'ABNB': 'buy&hold: -8.71%, strategy: -27.07%, alpha: -18.36%', 'GOOGL': 'buy&hold: 47.05%, strategy: -1.63%, alpha: -48.68%', 'GOOG': 'buy&hold: 47.40%, strategy: -6.42%, alpha: -53.83%', 'AMZN': 'buy&hold: 48.87%, strategy: 74.92%, alpha: 26.04%', 'AEP': 'buy&hold: 6.57%, strategy: -13.57%, alpha: -20.14%', 'AMGN': 'buy&hold: 22.72%, strategy: 24.20%, alpha: 1.48%', 'ADI': 'buy&hold: 47.93%, strategy: -7.58%, alpha: -55.51%', 'AAPL': 'buy&hold: 63.90%, strategy: 38.56%, alpha: -25.35%', 'AMAT': 'buy&hold: 33.95%, strategy: -6.72%, alpha: -40.67%', 'APP': 'buy&hold: 604.59%, strategy: 1301.95%, alpha: 697.36%', 'ARM': 'buy&hold: 99.74%, strategy: 93.71%, alpha: -6.03%', 'ASML': 'buy&hold: 19.70%, strategy: 6.44%, alpha: -13.27%', 'AZN': 'buy&hold: 12.17%, strategy: 6.75%, alpha: -5.41%', 'TEAM': 'buy&hold: -0.73%, strategy: 16.11%, alpha: 16.84%', 'ADSK': 'buy&ho

# Fucntions for asset screener

In [6]:
def pct_to_float(s: str) -> float:
    """
    Convert strings into decimals:
    """
    s = s.strip().replace(",", "").replace("%", "")
    return float(s) / 100.0

def results_dict_to_df(results: dict) -> pd.DataFrame:
    """
    returns DataFrame with columns bh, strat, alpha (decimals), bh_pct, strat_pct, alpha_pct
    """
    rows = []
    pat = re.compile(r"buy&hold:\s*([-0-9.,]+%)\s*,\s*strategy:\s*([-0-9.,]+%)\s*,\s*alpha:\s*([-0-9.,]+%)")

    for ticker, text in results.items():
        m = pat.search(text)
        if not m:
            raise ValueError(f"Could not parse line for {ticker}: {text}")

        bh_s, strat_s, alpha_s = m.groups()
        bh = pct_to_float(bh_s)
        strat = pct_to_float(strat_s)
        alpha = pct_to_float(alpha_s)

        rows.append({
            "ticker": ticker,
            "bh": bh,
            "strategy": strat,
            "alpha": alpha,
            "bh_pct": bh * 100,
            "strategy_pct": strat * 100,
            "alpha_pct": alpha * 100,
        })

    df = pd.DataFrame(rows).set_index("ticker")
    return df

def top_n(df: pd.DataFrame, rank_by: str = "alpha", n: int = 20, ascending: bool = False) -> pd.DataFrame:
    """
    rank_by: 'bh', 'strategy', or 'alpha'
    """
    if rank_by not in {"bh", "strategy", "alpha"}:
        raise ValueError("rank_by must be one of: 'bh', 'strategy', 'alpha'")

    out = df.sort_values(rank_by, ascending=ascending).head(n).copy()

    # Pretty formatting for display
    out["buy&hold"] = out["bh"].map(lambda x: f"{x:.2%}")
    out["strategy_ret"] = out["strategy"].map(lambda x: f"{x:.2%}")
    out["alpha_ret"] = out["alpha"].map(lambda x: f"{x:.2%}")

    return out[["buy&hold", "strategy_ret", "alpha_ret"]]

In [7]:
df = results_dict_to_df(summary_dict)

# Top b=20 by Total Return

In [ ]:
print('Top 20 by buy&hold (Total return of asset over period)')
print(top_n(df, rank_by="bh", n=20))


Top 20 by buy&hold (Gross return of asset over period)
       buy&hold strategy_ret alpha_ret
ticker                                
MSTR    631.11%       -5.92%  -637.02%
APP     604.59%     1301.95%   697.36%
PLTR    602.23%      165.41%  -436.82%
NVDA    485.43%      181.14%  -304.28%
AXON    390.44%      294.62%   -95.82%
AVGO    318.45%       81.54%  -236.91%
CEG     298.83%      237.15%   -61.68%
PDD     255.14%      -22.00%  -277.15%
META    206.07%      136.15%   -69.92%
NFLX    159.29%      145.75%   -13.54%
BKNG    140.96%       30.95%  -110.01%
TTD     118.17%       58.31%   -59.87%
CDNS    110.85%        8.06%  -102.79%
DASH    110.29%       24.05%   -86.24%
PANW    102.16%       30.87%   -71.28%
PCAR    101.59%       62.26%   -39.34%
ARM      99.74%       93.71%    -6.03%
CTAS     96.33%       81.02%   -15.31%
KLAC     95.09%        6.48%   -88.60%
ISRG     93.32%       66.64%   -26.68%


# Top 20 by Strategy Return

In [9]:
print('Top 20 by strategy return')
print(top_n(df, rank_by="strategy", n=20))  # Top 20 by strategy return

Top 20 by strategy return
       buy&hold strategy_ret alpha_ret
ticker                                
APP     604.59%     1301.95%   697.36%
AXON    390.44%      294.62%   -95.82%
CEG     298.83%      237.15%   -61.68%
NVDA    485.43%      181.14%  -304.28%
PLTR    602.23%      165.41%  -436.82%
NFLX    159.29%      145.75%   -13.54%
META    206.07%      136.15%   -69.92%
CRWD     81.38%      119.71%    38.33%
ARM      99.74%       93.71%    -6.03%
AVGO    318.45%       81.54%  -236.91%
CTAS     96.33%       81.02%   -15.31%
AMZN     48.87%       74.92%    26.04%
ISRG     93.32%       66.64%   -26.68%
PCAR    101.59%       62.26%   -39.34%
TTD     118.17%       58.31%   -59.87%
SHOP     92.13%       49.93%   -42.20%
CCEP     79.32%       44.07%   -35.25%
AAPL     63.90%       38.56%   -25.35%
COST     76.03%       37.85%   -38.18%
TSLA     51.08%       35.85%   -15.24%


# Top 20 by Alpha (outperformance)

In [10]:
print('Top 20 by alpha (outperformance of gross return by strategy return)')
print(top_n(df, rank_by="alpha", n=20))     # Top 20 by alpha

Top 20 by alpha (outperformance of gross return by strategy return)
       buy&hold strategy_ret alpha_ret
ticker                                
APP     604.59%     1301.95%   697.36%
DXCM    -25.24%       22.75%    47.99%
CRWD     81.38%      119.71%    38.33%
AMZN     48.87%       74.92%    26.04%
PYPL    -15.04%       10.66%    25.71%
INTC    -52.12%      -34.56%    17.56%
TEAM     -0.73%       16.11%    16.84%
CHTR    -39.78%      -25.26%    14.52%
GEHC      3.14%       11.22%     8.08%
CMCSA   -10.75%       -3.28%     7.46%
KHC      -8.26%       -0.96%     7.30%
BIIB    -22.52%      -17.18%     5.34%
IDXX    -17.79%      -14.88%     2.92%
AMGN     22.72%       24.20%     1.48%
WBD     -57.75%      -57.87%    -0.12%
CTSH     -8.71%       -9.22%    -0.51%
MDLZ      5.06%        3.25%    -1.81%
KDP      -8.07%      -10.35%    -2.27%
CSX      -2.53%       -5.39%    -2.87%
CSCO     16.18%       11.44%    -4.74%
